In [1]:
import sys
sys.path.append(
    "/usr/lib/spark/python"
)
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import year, month, dayofmonth

In [15]:
spark = SparkSession.builder.appName('g1').getOrCreate()

In [16]:
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.access.key", "YCAJEQ6hoybux3TZsjsbpURg4")
hadoop_conf.set("fs.s3a.secret.key", "YCPhC9mxk9Or5WIqcbyvUMpogn2UbWyHlRedILTb")
hadoop_conf.set("fs.s3a.endpoint", "storage.yandexcloud.net")

In [17]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, TimestampType

schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("tx_datetime", TimestampType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("terminal_id", IntegerType(), True),
    StructField("tx_amount", DoubleType(), True),
    StructField("tx_time_seconds", IntegerType(), True),
    StructField("tx_time_days", IntegerType(), True),
    StructField("tx_fraud", IntegerType(), True),
    StructField("tx_fraud_scenario", IntegerType(), True)
])

In [9]:
df = spark.read.csv("s3://mlops-homework-rvsmsjim/",
                    header=False,
                    pathGlobFilter="*.txt",
                    schema=schema, comment='#',
                    mode='DROPMALFORMED')

In [10]:
df.show()

+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|             0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|             1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|             2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|             3|2019-08-22 07:21:33|          0|          0|    80.41|          26493|           0|       0|                0|
|             4|2019-08-22 09:06:17|          1|        981|   102.83|          32777|           0|       0|   

In [ ]:
%%time
df.write.parquet("s3a://mlops-homework-rvsmsjim/dataset.parquet", mode="overwrite")

In [18]:
df1 = spark.read.parquet("s3a://mlops-homework-rvsmsjim/dataset.parquet")

In [6]:
%%time
df1.count()

CPU times: user 307 µs, sys: 4.01 ms, total: 4.31 ms
Wall time: 21.3 s


1319707844

In [32]:
from pyspark.sql.functions import isnan, when, count, col
df1.select([count(when(isnan(c), c)).alias(c) for c, t in df1.dtypes if t != "timestamp"]).show()

+--------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|             0|          0|          0|        0|              0|           0|       0|                0|
+--------------+-----------+-----------+---------+---------------+------------+--------+-----------------+



In [9]:
from pyspark.sql.functions import year, month, dayofmonth

In [7]:
df1.where((year(df1.tx_datetime) < 2018) | (year(df1.tx_datetime) > 2024)).count()

0

In [8]:
df1.where((month(df1.tx_datetime) < 1) | (month(df1.tx_datetime) > 12)).count()

0

In [9]:
df1.where((dayofmonth(df1.tx_datetime) < 1) | (dayofmonth(df1.tx_datetime) > 31)).count()

0

In [10]:
df1.where(df1.transaction_id <= 0).count()

1

In [11]:
df1.where(df1.customer_id < 0).count()

10128

In [12]:
df1.where(df1.terminal_id < 0).count()

0

In [13]:
df1.where(df1.tx_amount <= 0).count()

24806

In [14]:
df1.where(df1.tx_time_seconds <= 0).count()

3

In [15]:
df1.select('tx_fraud').distinct().collect()

[Row(tx_fraud=1), Row(tx_fraud=0)]

In [16]:
df1.select('tx_fraud_scenario').distinct().collect()

[Row(tx_fraud_scenario=1),
 Row(tx_fraud_scenario=3),
 Row(tx_fraud_scenario=2),
 Row(tx_fraud_scenario=0)]

In [7]:
df1.select('transaction_id').distinct().count()

1319706183

In [20]:
df2 = (
df1
    .where((year(df1.tx_datetime) >= 2018) | (year(df1.tx_datetime) <= 2024))
    .where((month(df1.tx_datetime) >= 1) | (month(df1.tx_datetime) <= 12))
    .where((dayofmonth(df1.tx_datetime) >= 1) | (dayofmonth(df1.tx_datetime) <= 31))
    .where(df1.customer_id >= 0)
    .where(df1.terminal_id > 0)
    .where(df1.tx_amount >= 0)
    .where(df1.tx_time_seconds >= 0)
    .where((df1.tx_fraud == 0) | (df1.tx_fraud == 1))
    .where((df1.tx_fraud_scenario == 0) | (df1.tx_fraud_scenario == 1) | (df1.tx_fraud_scenario == 2) | (df1.tx_fraud_scenario == 3))
    .dropDuplicates(['transaction_id'])
    .count()
)

1262672732

In [14]:
spark.stop()